In [6]:
import os
import pandas as pd
import  matplotlib.pyplot as plt

from transformers import AutoTokenizer, AutoModel
from transformers import DefaultDataCollator
from datasets import load_dataset

from InitialParameter import InitialParameter

import sys
sys.path.append('./mymodelkit/')
from mymodelkit import TextDataAugmentation, jumanTextAlignment

In [7]:
parameter = InitialParameter()
parameter

InitialParameter(implemnt_len=1, train_data_dir='./data/train', train_list=['sokushin_train.xlsx'], test_data_dir='./data/test', test_list=['sokushin_test.xlsx'], save_dir='/home/wakasugi/model_create/MultiFacilitatedAxis/result', save_list=<class 'list'>, use_model_list=['nlp-waseda/roberta-base-japanese'], text_column_name='ツイート', label_column_name='判定結果', label_list=['推奨', '行動抑制', '励まし', '願望'], JaToEn_label={'推奨': 'recommend', '行動抑制': 'suppression', '励まし': 'encouragement', '願望': 'desire'}, JaToEn_column={'ツイート': 'tweet', '判定結果': 'label'}, max_length=256, training_args=TrainingArguments(
_n_gpu=2,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=False,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train

# テキスト拡張

In [13]:
ReadFile = './original/sokushinラベル確定_result.xlsx'
OutputFile = 'sokushin_jumanAndAugmented.xlsx'

In [3]:
all_data_pd = pd.read_excel(ReadFile)
all_data_pd

,ツイート,判定結果
0,お帰りなさい~ ご無事で何より お疲れさまでした 台風に勝ったw ≡ ~ 一日ずれた...,その他
1,おはようございます週末関東も台風の影響があるようです。最近、雨や風での被害が多いのに台風がま...,推奨
2,そっちの方は台風の影響受けなさそうなのかな?気を付けて行ってらっしゃい~! ( ),その他
3,【台風被害に伴う施設開放終了のお知らせ】10月15日をもちまして、施設の開放を終了しました。...,推奨
4,ありがとうございます飲料水や食料品、電池などの備蓄は南海トラフに備えてしてあったのですが、断...,推奨
...,...,...
9995,ありがとうございます!私や家族は大丈夫ですが近所の方や同じ区内で困ってる方もいると思います...,願望
9996,09/07 05:08 発表 [令和元年台風第13号に関する奄美地方気象情報] 奄美地方では...,推奨
9997,今回の件台風では千葉の人たち身の安全を第一に、なんとか乗り切って下さいね♀♀,その他
9998,1/2【北海道地方 気象情報 2019年10月12日 16:49】北海道地方では、13日は...,推奨


In [4]:
textaugmentation = TextDataAugmentation.TextAugmentation(parameter)

In [5]:
#テキスト拡張を行うデータ(DataFrame)を取り出す
target_augmentation = pd.DataFrame()
not_target_augmentation = pd.DataFrame() #拡張対象でないDataFrameを保存
for target_ in ['行動抑制', '励まし', '願望']:  #このリストに含まれているラベルを持つSeriesをtarget_augmentationに保存
    tmp_augmentation = all_data_pd[all_data_pd['判定結果'].str.contains(target_)]
    target_augmentation = pd.concat([target_augmentation, tmp_augmentation], axis=0)
    
else:
    #上記のfor文で条件を満たさなかったindexのリストを作成
    non_target = [True if i not in target_augmentation.index.to_list() else False for i in range(len(all_data_pd))]
    not_target_augmentation = all_data_pd.iloc[non_target, :] 
    target_augmentation.reset_index(drop=True,inplace=True)
    not_target_augmentation.reset_index(drop=True,inplace=True)
target_augmentation

,ツイート,判定結果
0,西表島と石垣島に住んでいたときに風速70mの台風を経験しました。経験の中でこれはやっていた方...,推奨|行動抑制
1,徐々に風雨が強まっています。外出はできるだけ控えてください。また、各自治体ではすでに多くの...,推奨|行動抑制
2,【台風接近について】台風19号の進路にあたる地域では不要不急の外出は控え、注意報・警報の発表...,推奨|行動抑制
3,実家より送られてきた。 雨も風も強くなってきてる。 満潮の時間なのか高潮にも注意。 家でじっ...,推奨|行動抑制
4,【お知らせ】 台風の影響により営業困難だと判断し、19日(月)のバニーガールイベントは中止と...,推奨|行動抑制
...,...,...
1544,千葉県東金市の住民の皆さん大丈夫ですか?今朝から連絡も取れず安否が心配です飲み物や食べ物もほ...,願望
1545,"""関東の1都6県のファミリーマート約5500店で令和元年台風15号被害支援金募金の受付を開始...",願望
1546,#千葉#台風#雨漏り友人や家族、職場の方が雨漏りで困っていたら教えてあげてください,願望
1547,静岡の台風被害の状況をTwitterで知ったけど断水に停電し続けているのか。 情報がわからな...,願望


In [9]:
#拡張開始
augmented_pd = textaugmentation.run(target_augmentation)
augmented_pd

augment: 100%|████████████████████████████████████| 4/4 [00:00<00:00, 21.43it/s]


augment: 100%|████████████████████████████████████| 4/4 [00:00<00:00,  7.97it/s]


augment: 100%|████████████████████████████████████| 4/4 [00:00<00:00, 15.09it/s]


augment: 100%|████████████████████████████████████| 4/4 [00:00<00:00,  5.35it/s]


augment: 100%|████████████████████████████████████| 4/4 [00:00<00:00, 10.33it/s]


augment: 100%|████████████████████████████████████| 4/4 [00:00<00:00, 10.97it/s]


augment: 100%|████████████████████████████████████| 4/4 [00:00<00:00, 16.79it/s]


augment: 100%|████████████████████████████████████| 4/4 [00:00<00:00,  9.43it/s]


In [10]:
augmented_pd

,ツイート,判定結果
0,西表島と石垣島に住んでいたときに風速70m履行の台風を経験しました。経験の中でこれはやってい...,推奨|行動抑制
1,西表島と対策て住んでいたときに風速70mの台風て経験しましに。経験の中でこれにやっていた方が...,推奨|行動抑制
2,西表島と石垣島に住んでいたときに風速70m台風を経験しました。経験中でこれはやっていた方がい...,推奨|行動抑制
3,西表島と石垣島に住んでいたときに風速70mの台風を経験しました。の中でこれはやっていた方がい...,推奨|行動抑制
4,西表島と石垣島に住んでいたときに風速70mの台風を経験しました。経験の中でこれはやっていた方...,推奨|行動抑制
...,...,...
7740,ありがとうございます!私や家族ドンとは大丈夫ですが近所永らえるの方や同じ区内で困ってる方も...,願望
7741,ありがとうございます!私や世帯は大丈夫ですが附近の方向や同じ区内で困ってる方向もいると思い...,願望
7742,ありがとうございます!私や家族は大丈夫ですが近所の方や同じ区内で困ってる方もいると思います...,願望
7743,ありがとうございます!私や家族は大丈夫ですが近所の方や区内で困ってる方もいると思います。今...,願望


In [11]:
all_juman = pd.concat([augmented_pd, not_target_augmentation], axis=0)
all_juman

,ツイート,判定結果
0,西表島と石垣島に住んでいたときに風速70m履行の台風を経験しました。経験の中でこれはやってい...,推奨|行動抑制
1,西表島と対策て住んでいたときに風速70mの台風て経験しましに。経験の中でこれにやっていた方が...,推奨|行動抑制
2,西表島と石垣島に住んでいたときに風速70m台風を経験しました。経験中でこれはやっていた方がい...,推奨|行動抑制
3,西表島と石垣島に住んでいたときに風速70mの台風を経験しました。の中でこれはやっていた方がい...,推奨|行動抑制
4,西表島と石垣島に住んでいたときに風速70mの台風を経験しました。経験の中でこれはやっていた方...,推奨|行動抑制
...,...,...
8464,おはようございます台風が、また来ますね天気の悪い日には、美術館博物館へ。陶磁美術館へようこそ...,推奨
8465,09/07 05:08 発表 [令和元年台風第13号に関する奄美地方気象情報] 奄美地方では...,推奨
8466,今回の件台風では千葉の人たち身の安全を第一に、なんとか乗り切って下さいね♀♀,その他
8467,1/2【北海道地方 気象情報 2019年10月12日 16:49】北海道地方では、13日は...,推奨


In [16]:
jumantextalignment = jumanTextAlignment.jumanTextAlignment(parameter)
juman_augmented_pd = jumantextalignment.run(all_juman)
juman_augmented_pd

,ツイート,判定結果
0,西表 島 と 石垣 島 に 住んで いた とき に 風速 履行 の 台風 を 経験 し まし...,推奨|行動抑制
1,西表 島 と 対策 て 住んで いた とき に 風速 の 台風 て 経験 し ましに 経験 ...,推奨|行動抑制
2,西表 島 と 石垣 島 に 住んで いた とき に 風速 台風 を 経験 し ました 経験 ...,推奨|行動抑制
3,西表 島 と 石垣 島 に 住んで いた とき に 風速 の 台風 を 経験 し ました の...,推奨|行動抑制
4,西表 島 と 石垣 島 に 住んで いた とき に 風速 の 台風 を 経験 し ました 経...,推奨|行動抑制
...,...,...
8464,おはよう ございます 台風 が また 来 ます ね 天気 の 悪い 日 に は 美術 館 博...,推奨
8465,発表 令和 元年 台風 第 号 に 関する 奄美 地方 気象 情報 奄美 地方 で は うね...,推奨
8466,今回 の 件 台風 で は 千葉 の 人 たち 身 の 安全 を 第 一 に なんとか 乗り...,その他
8467,北海道 地方 気象 情報 年月日 北海道 地方 で は 日 は 台風 第 号 の 影響 に ...,推奨


In [17]:
jumantextalignment.save(juman_augmented_pd,'data',OutputFile)

'data/jumanTextAlignment_sokushin_jumanAndAugmented.xlsx'

# Dataset作成

In [1]:
from sklearn.model_selection import train_test_split
import os
import pandas as pd

In [2]:
ReadFile = 'sokushin_jumanAndAugmented.xlsx' #読み込むデータはdataフォルダーにあることを前提とする

Train_rate = 0.8

In [3]:
all_data_pd = pd.read_excel(os.path.join('data',ReadFile))

#all_data_pd.iloc[:, -1] = all_data_pd.iloc[:, -1].apply(lambda x: x.split('|'))
#all_data_pd.iloc[:, -1] = all_data_pd.iloc[:, -1].apply(lambda x: [parameter.JaToEn_label.get(word) if parameter.JaToEn_label.get(word)!= None else ''  for word in x ])
#all_data_pd = all_data_pd.rename(columns=parameter.JaToEn_column)
all_data_pd

,ツイート,判定結果
0,西表 島 と 石垣 島 に 住んで いた とき に 風速 履行 の 台風 を 経験 し まし...,推奨|行動抑制
1,西表 島 と 対策 て 住んで いた とき に 風速 の 台風 て 経験 し ましに 経験 ...,推奨|行動抑制
2,西表 島 と 石垣 島 に 住んで いた とき に 風速 台風 を 経験 し ました 経験 ...,推奨|行動抑制
3,西表 島 と 石垣 島 に 住んで いた とき に 風速 の 台風 を 経験 し ました の...,推奨|行動抑制
4,西表 島 と 石垣 島 に 住んで いた とき に 風速 の 台風 を 経験 し ました 経...,推奨|行動抑制
...,...,...
16209,おはよう ございます 台風 が また 来 ます ね 天気 の 悪い 日 に は 美術 館 博...,推奨
16210,発表 令和 元年 台風 第 号 に 関する 奄美 地方 気象 情報 奄美 地方 で は うね...,推奨
16211,今回 の 件 台風 で は 千葉 の 人 たち 身 の 安全 を 第 一 に なんとか 乗り...,その他
16212,北海道 地方 気象 情報 年月日 北海道 地方 で は 日 は 台風 第 号 の 影響 に ...,推奨


In [4]:
a_train, a_test = train_test_split(all_data_pd,train_size=Train_rate)
a_train

,ツイート,判定結果
12476,今日 も 一 日 お 疲れ 様 でした 台風 の 影響 で 離れて る 地域 でも 雨 風 ...,推奨
6254,なんか テレビ の 他人事 感 凄い な とりあえず 南房総 報道 し ときゃ いい んでし...,願望
1983,皆さん 台風 大丈夫 で しか 初めて お家 の 電気 が 少し 消えて びっくり し まし...,励まし
6962,今回 に 台風 は 本当の 悪い 子 が 人 いろんな 迷惑 がって る 反省 して ほしい,願望
4500,台風 号 被害 を られた 方々 に 心 より お 見舞い 申し上げ ます さん で は を...,願望
...,...,...
13410,台風 台風 な のです 低 気圧 が 気 を 付けて ください ませ,推奨
15447,通電 火災 知って る 去年 の 台風 の 時 一応 電気 屋 さんや から 会社 の 人 ...,推奨
2527,お は シグ ねー ⊃ アワアワ 今日 一体 育祭 ので 休み です が 吹奏 楽部 に 休...,推奨|励まし
11034,台風 も 来て ます しお気 を つけて お 帰り 下さい ませ ゚ Д ゚,推奨


In [8]:
for trafile,tesfile in zip(parameter.train_list, parameter.test_list):
    a_train.to_excel(os.path.join(parameter.train_data_dir, trafile), index=None)
    a_test.to_excel(os.path.join(parameter.test_data_dir, tesfile), index=None)